- 사회적 불안지수 만들어야함
- 이혼율, 자살률, 1인가구율 등 단위와 범위가 다른 변수들을 0~1 사이의 같은 스케일로 맞추기 위해 정규화를 진행함
- 연관성에 대해서는 상관관계를 분석함
- 자치구 기준으로 병합
- 봉사활동 지표는 역정규화 처리
- mean으로 정규화된 지표를 계산한다. 

In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

# 필요한 패키지
import os

In [2]:
# 현재 경로 확인
os.getcwd()


'c:\\Users\\admin\\Desktop\\mid_project\\MidProject\\ipynbTest\\Junhyeok'

In [3]:

# 상위 폴더로 이동
os.chdir('../../')

In [4]:
# 현재 경로 확인
os.getcwd()


'c:\\Users\\admin\\Desktop\\mid_project\\MidProject'

In [ ]:
# 예시 파일 경로들
df_divorce = pd.read_csv('data/processed/혼인이혼_동별.csv')          # 이혼율
df_single = pd.read_csv('data/processed/1인가구합.csv')         # 1인가구
df_dropout = pd.read_csv('data/processed/2023 구별 학업 중단율.csv')       # 학업중단율
df_suicide = pd.read_csv('data/processed/자살율통계_서울시울시2023.csv')          # 자살률
df_volunteer = pd.read_csv('data/processed/2023_봉사활동_등록현황.csv')       # 봉사활동 수

In [16]:
#데이터를 확인한다.
df_divorce.head()

,자치구별,2023,2023.1
0,자치구별,혼인,이혼
1,종로구,440,199
2,중구,595,197
3,용산구,1002,312
4,성동구,1280,340


In [17]:
# 컬럼 이름을 바꿔준다.
df_divorce = df_divorce.rename(columns={'2023': '혼인',
                                        '2023.1' : '이혼'})
# 그리고 0번 raw는 필요없기 때문에 빼고 만들들겠다. 
df_divorce = df_divorce[(df_divorce['자치구별'] != '자치구별')]

# 데이터프레임을 확인한다. 
df_divorce.head()

,자치구별,혼인,이혼
1,종로구,440,199
2,중구,595,197
3,용산구,1002,312
4,성동구,1280,340
5,광진구,1361,450


In [18]:
# 인덱스 초기화
df_divorce = df_divorce.reset_index(drop=True)

In [19]:
# 우선 혼인 컬럼의 경우 필요 없기 때문에 없애준다.
df_divorce.drop('혼인', axis=1, inplace=True) 
df_divorce.head()

,자치구별,이혼
0,종로구,199
1,중구,197
2,용산구,312
3,성동구,340
4,광진구,450


In [20]:
# 이후 인구수 데이터를 위해 봉사활동 데이터를 가져오겠다. 
# 인구소계, 자치구구 컬럼, 등록률만 가져온 데이터프레임을 만든다. 
volunteer_df = df_volunteer[['자치구', '인구소계', '등록률 (%)']]
# 가져와서 등록률을 알아보기 쉽게 하기 위해 이름을 봉사활동 등록률으로 변경하도록 하겠다. 
volunteer_df = volunteer_df.rename(columns={'등록률 (%)': '봉사활동 등록률'})
# 그리고 데이터를 확인하도록 하겠다. 
volunteer_df.head()

,자치구,인구소계,봉사활동 등록률
0,종로구,139417,45.09
1,중구,121312,33.53
2,용산구,213151,26.23
3,성동구,277361,27.77
4,광진구,335554,27.17


In [21]:
# 우선 inner join으로 자치구가 있는 raw만 묶어서 데이터프레임으로 만들겠다. 
# 봉사활동 데이터의 경우 컬럼명이 자치구이므로 자치구별로 바꿔준 뒤 진행하겠다. 
volunteer_df.rename(columns={'자치구': '자치구별'}, inplace=True) 
merged1_df = df_divorce.merge(volunteer_df, on='자치구별', how='inner')
merged1_df.head()

,자치구별,이혼,인구소계,봉사활동 등록률
0,종로구,199,139417,45.09
1,중구,197,121312,33.53
2,용산구,312,213151,26.23
3,성동구,340,277361,27.77
4,광진구,450,335554,27.17


In [24]:
# 이혼율을 계산하는 컬럼을 만들어주겠다. 
# 문자열로 이루어진 데이터라 계산이 되지 않아 정수값으로 변환한 뒤 진행한다. 
# 문자열을 숫자로 변환
merged1_df['이혼'] = merged1_df['이혼'].astype(int)
merged1_df['인구소계'] = merged1_df['인구소계'].astype(int)
# 이혼율을 계산한다. 
merged1_df['이혼율'] = (merged1_df['이혼'] / merged1_df['인구소계']) * 1000
# 그리고 둘째자리에서 반올림하겠다. 
merged1_df['이혼율'] = merged1_df['이혼율'].round(2)
# 데이터를 확인한다. 
merged1_df

,자치구별,이혼,인구소계,봉사활동 등록률,이혼율
0,종로구,199,139417,45.09,1.43
1,중구,197,121312,33.53,1.62
2,용산구,312,213151,26.23,1.46
3,성동구,340,277361,27.77,1.23
4,광진구,450,335554,27.17,1.34
5,동대문구,415,341149,26.51,1.22
6,중랑구,599,382155,28.52,1.57
7,성북구,522,425602,25.13,1.23
8,강북구,499,288113,21.90,1.73
9,도봉구,473,306948,26.68,1.54


In [25]:
# 이제 이혼건수는 필요없으니 없애주도록 하겠다.
merged1_df.drop('이혼', axis=1, inplace=True)
merged1_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율
0,종로구,139417,45.09,1.43
1,중구,121312,33.53,1.62
2,용산구,213151,26.23,1.46
3,성동구,277361,27.77,1.23
4,광진구,335554,27.17,1.34


In [27]:
# 첫번째 합쳐진 데이터를 csv 파일로 내보낸다. 
merged1_df.to_csv('members/조준혁/합친거1.csv', encoding='utf-8-sig', index=True)

In [29]:
# 다른 데이터프레임도 merge하도록 하겠다. 
# 우선 데이터를 확인한다
df_single.head()

,Unnamed: 0,자치구별,1인가구수(남자),1인가구수(여자)
0,0,강남구,33217,43592
1,1,강동구,29123,33820
2,2,강북구,24116,28543
3,3,강서구,47105,57404
4,4,관악구,79518,71227


In [30]:
df_single.drop('Unnamed: 0', axis=1, inplace=True) 
df_single.head()

,자치구별,1인가구수(남자),1인가구수(여자)
0,강남구,33217,43592
1,강동구,29123,33820
2,강북구,24116,28543
3,강서구,47105,57404
4,관악구,79518,71227


In [31]:
# 여자와 남자의 수는 쓸모 있을 것 같다. 그러므로 새로운 컬럼을 만들어서 합쳐주겠다. 
# '남자1인가구' + '여자1인가구' → '1인가구합' 생성
df_single['1인가구수(남녀총합)'] = df_single['1인가구수(남자)'] + df_single['1인가구수(여자)']

In [32]:
df_single.head()

,자치구별,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합)
0,강남구,33217,43592,76809
1,강동구,29123,33820,62943
2,강북구,24116,28543,52659
3,강서구,47105,57404,104509
4,관악구,79518,71227,150745


In [34]:
merged2_df = merged1_df.merge(df_single, on='자치구별', how='inner')
merged2_df

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합)
0,종로구,139417,45.09,1.43,14040,15294,29334
1,중구,121312,33.53,1.62,13574,14719,28293
2,용산구,213151,26.23,1.46,19436,23046,42482
3,성동구,277361,27.77,1.23,23464,24647,48111
4,광진구,335554,27.17,1.34,34355,37604,71959
5,동대문구,341149,26.51,1.22,34960,35999,70959
6,중랑구,382155,28.52,1.57,32454,34452,66906
7,성북구,425602,25.13,1.23,30867,38281,69148
8,강북구,288113,21.90,1.73,24116,28543,52659
9,도봉구,306948,26.68,1.54,18400,23125,41525


In [35]:
# csv로 내보낸다.
merged2_df.to_csv('members/조준혁/합친거2.csv', encoding='utf-8-sig', index=True)